# Energy gap vs V For Periodic Boundary Condition

In [ ]:
using ITensors, ITensorMPS, Plots

function run_dmrg_ll(N::Int, t::Float64, V::Float64, nsweeps::Int, maxdim::Vector{Int}, cutoff::Float64, noise::Float64, weight::Float64)
    # Define fermionic sites
    sites = siteinds("Fermion", N; conserve_qns = true)

    # Construct ll Hamiltonian 
    os = OpSum()
    for j in 1:N-1
        os += -t, "Cdag", j, "C", j + 1
        os += -t, "Cdag", j + 1, "C", j
        os += -V, "N", j, "N", j + 1  
    end

    # Periodic boundary conditions
    os += -t, "Cdag", 1, "C", N
    os += -t, "Cdag", N, "C", 1
    os += -V, "N", 1, "N", N    
    

    H = MPO(os , sites)  # Hamiltonian with interaction term

    # Compute ground state
    state = [isodd(j) ? "Occ" : "Emp" for j in 1:N]  # Half-filling

    psi0_init = randomMPS(sites, state)
    energy_ground, psi_ground = dmrg(H, psi0_init; nsweeps, maxdim, cutoff, noise)

    # Compute first excited state
    psi1_init = randomMPS(sites, state)
    orthogonalize!(psi1_init, 1)  # Ensure a different starting state

    energy_excited, psi_excited = dmrg(H, [psi_ground], psi1_init; nsweeps, maxdim, cutoff, noise, weight)

    # Check convergence
    H2 = apply(H, H)  # Hamiltonian squared
    E = inner(psi_ground, H, psi_ground) # Expectation value of H
    var = inner(psi_ground, H2, psi_ground) - E^2  # Variance
    println(var)  # Display variance

    energy_gap = energy_excited - energy_ground

    println("V = $V, Ground Energy: $energy_ground, Excited Energy: $energy_excited, Gap: $energy_gap")
    flush(stdout)
    return energy_gap
end

# Parameters
N = 64       # Number of sites
t = 1.0         # Hopping strength
V_values = collect(-3.6:0.4:2.0)  # Interaction strength
nsweeps = 10
maxdim= [10,20,50,100, 200]
cutoff = 1E-10 
noise = 1E-10
weight = 2.0  # Increased weight to ensure excited state separation

# Compute energy gaps for different V values
energy_gaps = [run_dmrg_ll(N, t, V, nsweeps, maxdim, cutoff, noise, weight) for V in V_values]

# Plot energy gap variation with V
plot(V_values, energy_gaps, xlabel="V", ylabel="Energy Gap", title="Energy Gap vs Interaction Strength V",marker=:o,  linewidth=1)
# Plot Number Density ⟨C_i^† C_i⟩ vs Site Index
plot(1:N, number_density, xlabel="Site Index", ylabel="Number Density ⟨C_i^† C_i⟩", title="Number Density vs Site Index (V = $V)", marker=:o, linewidth=2)

savefig("ll_interacting_engap_plot.png")

println("Energy gap values: ", energy_gaps)


# check number density at V = - 3.5 for Periodic Boundary Condition

In [60]:
using ITensors, ITensorMPS, Plots

function run_dmrg_ll(N::Int, t::Float64, V::Float64, nsweeps::Int, maxdim::Vector{Int}, cutoff::Float64, noise::Float64)
    # Define fermionic sites
    sites = siteinds("Fermion", N; conserve_qns = true)

    # Construct SSH Hamiltonian with interaction
    os = OpSum()
    for j in 1:N-1
        # Hopping term
        os += -t, "Cdag", j, "C", j + 1
        os += -t, "Cdag", j + 1, "C", j

        # Interaction term: V * n_j * n_{j+1}
        os += -V, "N", j, "N", j + 1
    end

    # Periodic boundary conditions
    os += -t, "Cdag", 1, "C", N
    os += -t, "Cdag", N, "C", 1
    os += -V, "N", 1, "N", N

    H = MPO(os, sites)

    # Initial MPS state (half-filling)
    state = [isodd(j) ? "Occ" : "Emp" for j in 1:N]
    psi0_init = randomMPS(sites, state)

    # Compute ground state using DMRG
    energy_ground, psi_ground = dmrg(H, psi0_init; nsweeps, maxdim, cutoff, noise=noise)

    # Compute number density ⟨C_i^† C_i⟩
    number_density = [inner(psi_ground, apply(op("N", sites[j]), psi_ground)) for j in 1:N]

    return energy_ground, number_density
end

# Parameters
N = 32 # Number of sites
t = 1.0  # Hopping strength
V = -3.5 # Interaction strength (only checking for V = -3.5)
nsweeps = 5
maxdim = [20, 40, 50, 100, 200]  
cutoff = 1E-10 
noise =  1E-10  # Decaying noise schedule

# Run DMRG for V = -3.5
energy_ground, number_density = run_dmrg_ll(N, t, V, nsweeps, maxdim, cutoff, noise)

# Print ground-state energy
println("Ground-state energy for V = $V: $energy_ground")

# Plot Number Density ⟨C_i^† C_i⟩ vs Site Index
plot(1:N, number_density, xlabel="Site Index", ylabel="Number Density ⟨C_i^† C_i⟩", 
     title="Number Density vs Site Index (V = $V)", marker=:o, linewidth=2)

# Save plot
filename = "ll_number_density_V$(V).png"
savefig(filename)
println("Number density plot saved: $filename")


# Check for Open Boundary Condition

In [76]:
using ITensors, ITensorMPS, Plots

function run_dmrg_ll(N::Int, t::Float64, V::Float64, nsweeps::Int, maxdim::Vector{Int}, cutoff::Float64, noise::Float64, weight::Float64)
    # Define fermionic sites
    sites = siteinds("Fermion", N; conserve_qns = true)

    # Construct Luttinger liquid Hamiltonian
    os = OpSum()
    for j in 1:N-1
        os += -t, "Cdag", j, "C", j + 1
        os += -t, "Cdag", j + 1, "C", j
        os += -V, "N", j, "N", j + 1  
    end

    H = MPO(os , sites)  # Hamiltonian with interaction term

    # Initial state (half-filling)
    state = [isodd(j) ? "Occ" : "Emp" for j in 1:N]
    psi0_init = randomMPS(sites, state)

    # Compute ground state using DMRG
    energy_ground, psi_ground = dmrg(H, psi0_init; nsweeps, maxdim, cutoff, noise)

    # Compute first excited state
    psi1_init = randomMPS(sites, state)
    orthogonalize!(psi1_init, 1)

    energy_excited, psi_excited = dmrg(H, [psi_ground], psi1_init; nsweeps, maxdim, cutoff, noise, weight)

    # Check convergence (variance)
    H2 = apply(H, H)
    E = inner(psi_ground, H, psi_ground)
    var = inner(psi_ground, H2, psi_ground) - E^2

    if var < 1e-8
        println("Ground state converged with low variance: $var")
    else
        println("Warning: High variance $var, consider increasing sweeps or maxdim.")
    end

    # Compute energy gap
    energy_gap = energy_excited - energy_ground

    println("V = $V, Ground Energy: $energy_ground, Excited Energy: $energy_excited, Gap: $energy_gap")
    flush(stdout)

    # Compute number density ⟨C_i^† C_i⟩
    return energy_gap
end

# Parameters
N = 64     # Number of sites
t = 1.0      # Hopping strength
V_values = collect(-3.6:0.4:2.0)  # Interaction strength
nsweeps = 10
maxdim = [10, 20, 50, 100, 200]
cutoff = 1E-10 
noise = 1E-10
weight = 2.0  # Ensuring excited state separation

# Compute energy gaps and number densities for different V values
energy_gaps = Float64[]
number_densities = Dict()

for V in V_values
    gap = run_dmrg_ll(N, t, V, nsweeps, maxdim, cutoff, noise, weight)
    push!(energy_gaps, gap)
end

# Plot energy gap variation with V
plot(V_values, energy_gaps, xlabel="V", ylabel="Energy Gap", title="Energy Gap vs Interaction Strength V", marker=:o, linewidth=1)
savefig("ll_interacting_engap_plot_obc.png")



println("Energy gap values: ", energy_gaps)
